# Component

> TODO fill in description

In [1]:
#| default_exp component

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| hide
import nbdev; nbdev.nbdev_export()

In [4]:
#|export
import asyncio
from abc import ABC, abstractmethod
from typing import Type

import fbdev
from fbdev import BasePort, BaseInputPort, SingleInputPort, BaseOutputPort, SingleOutputPort, Packet
from fbdev.utils import Expando, StateHandler

In [5]:
#|export
__all__ = ['BaseComponent']

In [6]:
#|hide
show_doc(fbdev.component.ComponentPorts)

---

### ComponentPorts

>      ComponentPorts ()

*Initialize self.  See help(type(self)) for accurate signature.*

In [7]:
#|export
class ComponentPorts(Expando):
    def __init__(self):
        super().__init__(obj_name="ComponentPorts")
        self.idx = ()
        
    def _add(self,
             port:BasePort):
        if port.name in self._attrs:
            #TODO proper exception in PortContainer._add
            raise Exception(f"Port with name {port.name} already exists.")
        self.idx = tuple(list(self.idx) + [port])
        self._attrs[port.name] = port

In [8]:
#|hide
show_doc(fbdev.component.ComponentConfig)

---

### ComponentConfig

>      ComponentConfig ()

*Initialize self.  See help(type(self)) for accurate signature.*

In [9]:
#|export
class ComponentConfig(Expando):
    def __init__(self):
        super().__init__(obj_name="ComponentConfig")

In [10]:
#|hide
show_doc(fbdev.component.BaseComponent)

---

### BaseComponent

>      BaseComponent (node)

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [11]:
#|export
class BaseComponent(ABC):
    config_input_specs = [] #TODO the way these specs look like are a bit ugly
    input_specs = []
    output_specs = []
    
    def __init_subclass__(cls, **kwargs):
        """Prevents any subclass from defining an __init__ that accepts any other argument than 'self' and 'node'."""
        super().__init_subclass__(**kwargs)
        init_method = cls.__dict__.get('__init__', None)
        if init_method:
            from inspect import signature
            sig = signature(init_method)
            params = tuple([p.name for p in sig.parameters.values()])
            if params != ('self', 'node'):
                raise TypeError(f"{cls.__name__}.__init__() must only accept 'self' and 'node' as arguments.")
    
    def __init__(self, node):
        self._node = node
        self.configured = StateHandler(True) # Will be set to False if the check fails below
        self.config_inputs = ComponentPorts()
        self._config_input_min_sizes = {}
        self._config_input_max_sizes = {}
        self.config = Expando(obj_name="component.config")
        self.inputs = ComponentPorts()
        self.outputs = ComponentPorts()
        
        # Attach ports based on configuration, inputs, and outputs
        self.__attach_ports(self.config_input_specs, self.config_inputs, SingleInputPort, is_config=True)
        self.__attach_ports(self.input_specs, self.inputs, SingleInputPort)
        self.__attach_ports(self.output_specs, self.outputs, SingleOutputPort)

    def __attach_ports(self, port_specs, port_container, default_port_type, is_config=False):
        """Attaches ports to the specified container based on the given specifications."""
        for port_spec in port_specs:
            name = port_spec['name']
            port_type = port_spec['port_type'] if 'port_type' in port_spec else default_port_type
            port_kwargs = port_spec['port_kwargs'] if 'port_kwargs' in port_spec else {}
            
            port = port_type(name, **port_kwargs)
            port_container._add(port)
            port.attach(self, port_container.idx.index(port))
            
            if is_config:
                self._config_input_min_sizes[name] = port_spec['min_size'] if 'min_size' in port_spec else 1
                self._config_input_max_sizes[name] = port_spec['max_size'] if 'max_size' in port_spec else -1
                
                if 'default' in port_spec:
                    if 'default_is_list' in port_spec and port_spec['default_is_list']:
                        for val in port_spec['default']:
                            port_container[name].load(Packet(val))
                    else:
                        port_container[name].load(Packet(port_spec['default']))
                else:
                    self.configured.set(False)
            
    async def update_config(self):
        for port in self.config_inputs.values():
            satisfies_min = self._config_input_min_sizes[port.name] == -1 or port.size() >= self._config_input_min_sizes[port.name]
            satisfies_max = self._config_input_max_sizes[port.name] == -1 or port.size() <= self._config_input_max_sizes[port.name] 
            
            if satisfies_min and satisfies_max:
                self.config._attrs[port.name] = await port.receive()
                
        self.configured.set(True)
    
    def set_config(self, port_name, data):
        self.config_inputs[port_name].put(data)
    
    @abstractmethod
    async def execute(self):
        pass

In [12]:
class MyComponent(BaseComponent):
    config_input_specs = [{'name':'msg', 'default':'Hello'}] 
    input_specs = [{'name':'in1'}]
    output_specs = [{'name':'out1'}]
    
    async def execute(self):
        data = await self.inputs.in1.receive()
        await self.outputs.out1.put(f"{self.config.msg} {data}")
        
comp = MyComponent(node=None)
comp.inputs.in1.load(Packet("world!"))
await comp.update_config()
await comp.execute()
assert comp.outputs.out1.unload().get_data() == "Hello world!"

In [13]:
comp.config_inputs.msg.load(Packet("Goodbye"))
comp.inputs.in1.load(Packet("world!"))
await comp.update_config()
await comp.execute()
assert comp.outputs.out1.unload().get_data() == "Goodbye world!"